In [5]:
from collections.abc import Mapping
from getpass import getpass
from elasticsearch import Elasticsearch, helpers
from tqdm import tqdm

import zipfile
import pandas as pd
import json
import openai

In [ ]:
client = Elasticsearch(IP)

In [3]:
df = pd.read_csv('./NL_BO_SPECIES_MASTER_NEW_202112.csv', encoding='utf-8')

/var/folders/bl/fn6byh392k1frtmltpy4c4w00000gn/T/ipykernel_83059/1883839175.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./NL_BO_SPECIES_MASTER_NEW_202112.csv', encoding='utf-8')


In [6]:
df = df[df['IMAGE_URL'].isna()!=True]
df = df[df['ISBN_THIRTEEN_NO'].isna()!=True]
df = df[df['BOOK_INTRCN_CN'].isna()!=True]
df = df[df['TITLE_NM'].isna()!=True]
df = df.reset_index(drop=True)

df['TITLE_VECTOR'] = None
df['CONTENT_VECTOR'] = None

In [7]:
df

,SEQ_NO,ISBN_THIRTEEN_NO,VLM_NM,TITLE_NM,AUTHR_NM,PUBLISHER_NM,PBLICTE_DE,ADTION_SMBL_NM,PRC_VALUE,IMAGE_URL,BOOK_INTRCN_CN,KDC_NM,TITLE_SBST_NM,AUTHR_SBST_NM,TWO_PBLICTE_DE,INTNT_BOOKST_BOOK_EXST_AT,PORTAL_SITE_BOOK_EXST_AT,ISBN_NO,TITLE_VECTOR,CONTENT_VECTOR
0,6352228,9791156759270,NaN,너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이,이용신 (지은이),푸른숲,NaN,NaN,16000.0,https://image.aladin.co.kr/product/28415/8/cov...,2004년 방영한 애니메이션 &lt;달빛천사&gt;에서 주인공 루나(풀문) 역을 맡...,NaN,NaN,NaN,2021-12-03,NaN,NaN,NaN,None,None
1,6352229,9791168120877,NaN,일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이,차도하 (지은이),위즈덤하우스,NaN,NaN,15800.0,https://image.aladin.co.kr/product/28414/66/co...,“그러니 나는 말하고 싶은 것을 말하겠다”「침착하게 사랑하기」 차도하 시인 첫 에세...,NaN,NaN,NaN,2021-12-06,NaN,NaN,NaN,None,None
2,6352230,9791168120839,NaN,본격 한중일 세계사 12 - 임오군란과 통킹 위기,굽시니스트 (지은이),위즈덤하우스,NaN,NaN,14800.0,https://image.aladin.co.kr/product/28414/47/co...,한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다. 러시아의 세력 확장을...,NaN,NaN,NaN,2021-12-01,NaN,NaN,NaN,None,None
3,6352231,9791168120846,NaN,즉시 기분을 바꿔드립니다 - 신기하게 마음이 편해지는 응급 처방전,"올리비아 레메스 (지은이), 김잔디 (옮긴이)",위즈덤하우스,NaN,NaN,14000.0,https://image.aladin.co.kr/product/28414/30/co...,"누구에게나 기분 구급상자가 필요하다. 하나씩 하나씩 차근차근, 좋은 기분을 쌓고 건...",NaN,NaN,NaN,2021-12-01,NaN,NaN,NaN,None,None
4,6352232,9791168120747,NaN,"오늘도 리추얼 : 음악, 나에게 선물하는 시간",정혜윤 (지은이),위즈덤하우스,NaN,NaN,15000.0,https://image.aladin.co.kr/product/28413/80/co...,나다운 일상을 만드는 사람들의 이야기를 담은 『오늘도 리추얼』 시리즈가 위즈덤하우스...,NaN,NaN,NaN,2021-12-01,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82874,6372478,9791197569708,NaN,우리 없는 세상,"로빈 스티븐슨 (지은이), 이은주 (옮긴이), 황상민 (감수)",잔잔씨,NaN,NaN,15000.0,https://image.aladin.co.kr/product/28540/73/co...,작가는 사회복지사와 상담가로 오래 일했다. 그는 자살로 누군가를 잃었거나 스스로 자...,NaN,우리없는세상,"로빈스티븐슨지은이,이은주옮긴이,황상민감수",2021-12-20,Y,Y,NaN,None,None
82875,6372479,9791138804486,NaN,위대한 영혼의 미소,이한옥 (지은이),좋은땅,NaN,NaN,16000.0,https://image.aladin.co.kr/product/28511/70/co...,재외 작가의 장편소설. 인고와 눈물로 점철된 코리안 뉴요커들의 이민 역정을 다룬 이...,NaN,위대한영혼의미소,이한옥지은이,2021-12-08,Y,Y,NaN,None,None
82876,6372480,9791138804523,NaN,"잊지 말자, 나는 기적이라는 걸 - 늘 그랬던 것처럼, 신발 끈을 고쳐 맬 우리",정매일 (지은이),좋은땅,NaN,NaN,13000.0,https://image.aladin.co.kr/product/28548/88/co...,갑작스레 세상에 나 혼자 남겨진 느낌이 들 때가 있다. 주위가 암전되고 덩그러니 남...,NaN,"잊지말자,나는기적이라는걸늘그랬던것처럼,신발끈을고쳐맬우리",정매일지은이,2021-12-05,Y,Y,NaN,None,None
82877,6372481,9791197549335,NaN,전념 - 나와 세상을 바꾸는 힘에 관하여,"피트 데이비스 (지은이), 신유희 (옮긴이)",상상스퀘어,NaN,NaN,17800.0,https://image.aladin.co.kr/product/28542/24/co...,오랫동안 하나의 목적을 이루기 위해 전념한 다양한 영웅들의 사례도 제시하고 있다. ...,NaN,전념나와세상을바꾸는에관하여,"피트데이비스지은이,신유희옮긴이",2022-01-05,Y,Y,NaN,None,None


In [ ]:
import boto3
import json

bedrock = boto3.client('bedrock-runtime', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region)

def getEmbedding(text):
    request_body = json.dumps({
        "texts": [text],
        "input_type": "search_document",
        "truncate": "NONE"
    })

    # Bedrock API 호출
    response = bedrock.invoke_model(
        modelId="cohere.embed-multilingual-v3",
        body=request_body
    )

    # 응답 파싱
    response_body = json.loads(response['body'].read())
    embeddings = response_body['embeddings'][0]

    return embeddings

def getQueyrEmbedding(query):
    request_body = json.dumps({
        "texts": [query],
        "input_type": "search_query",
        "truncate": "NONE"
    })

    # Bedrock API 호출
    response = bedrock.invoke_model(
        modelId="cohere.embed-multilingual-v3",
        body=request_body
    )

    # 응답 파싱
    response_body = json.loads(response['body'].read())
    embeddings = response_body['embeddings'][0]

    return embeddings


In [ ]:
import pandas as pd
import numpy as np

# 진행 상황을 추적하기 위한 카운터
processed_count = 0

for idx, row in tqdm(df.iterrows()):
    title = row['TITLE_NM']
    content = row['BOOK_INTRCN_CN']
    
    title_vector = getEmbedding(title)
    content_vector = getEmbedding(content)
    # 벡터를 리스트로 변환하여 DataFrame에 저장
    df.at[idx, 'TITLE_VECTOR'] = title_vector
    df.at[idx, 'CONTENT_VECTOR'] = content_vector
    
    # 진행 상황을 출력 (선택사항)
    processed_count += 1
    if processed_count % 100 == 0:
        print(f"Processed {processed_count} rows")

print(df.head())

In [ ]:
df.to_csv('df_book.csv')

In [ ]:
index_mapping = {
    "mappings": {
        "properties": {
            "title_vector": {
                "type": "dense_vector",
                "dims": 1024,
                "index": True,
                "similarity": "cosine"
            },
            "content_vector": {
                "type": "dense_vector",
                "dims": 1024,
                "index": True,
                "similarity": "cosine"
            },
            "text": {"type": "text"},
            "title": {"type": "text"},
            "img_url": {"type": "keyword"},
            "vector_id": {"type": "long"}
        }
    }
}

client.indices.create(index="book_vector_index", body=index_mapping)

In [ ]:
def dataframe_to_bulk_actions(df):
    for index, row in df.iterrows():
        yield {
            "_index": 'book_vector_index',
            "_id": row['ISBN_THIRTEEN_NO'],
            "_source": {
                'url' : row["IMAGE_URL"],
                'title' : row["TITLE_NM"],
                'text' : row["BOOK_INTRCN_CN"],
                'title_vector' : json.loads(row["TITLE_VECTOR"]),
                'content_vector' : json.loads(row["CONTENT_VECTOR"]),
                'vector_id' : index
            }
        }

In [ ]:
start = 0
end = len(df)
batch_size = 100
for batch_start in range(start, end, batch_size):
    batch_end = min(batch_start + batch_size, end)
    batch_dataframe = df.iloc[batch_start:batch_end]
    actions = dataframe_to_bulk_actions(batch_dataframe)
    helpers.bulk(client, actions)

In [ ]:
diary = "내일은 중요한 면접이 있는 날이다. 잘 할 수 있을지 걱정된다. 준비는 열심히 했지만, 자꾸 불안한 마음이 든다. 오늘 밤 잠을 잘 수 있을지 모르겠다. 긍정적인 마인드를 가지려 노력해야겠다."
diary = "오늘 직장에서 너무 화가 났다. 내가 열심히 준비한 프로젝트를 동료가 자기 것인 양 발표했다. 나의 노력을 인정받지 못하는 것 같아 너무 분하고 억울하다. 이런 부당한 일이 또 일어나지 않도록 해야겠다"
query = f"당신은 심리 분석가처럼 행동합니다.\n또한 당신은 모든 책에 대한 해박한 지식을 가지고 있습니다.\n아래의 일기를 읽어보고 글쓴이에게 어울릴만한 책을 추천해주시기 바랍니다.\n\n### 일기 ###\n{diary}"
print(query)

question_embedding = getQueyrEmbedding(query)

In [ ]:
query = {
    "knn": {
        "query_vector": question_embedding,
        "k": 3,
        "field": "content_vector"  # This remains the same, as it refers to the field in your index
    },
    "_source": ["text", "title", "url"],
    "size": 10
}

response = client.search(
    index="book_vector_index",
    body=query
)

pretty_response(response)

top_hit_summary = response['hits']['hits'][0]['_source']['text']